# Bistable toggle switch model

This notebook implements the bistable toggle switch model. The particularity of it is that the propensities are given as custom functions and that the PMF has a bimodal behavior. 

Imports, set the accuracy to float64 and set the `qtt` flag (`True` if QTT format should be used).

In [ ]:
import torch as tn
import torchtt as tntt
import TTCME
import matplotlib.pyplot as plt 
import datetime
import numpy as np

tn.set_default_tensor_type(tn.DoubleTensor)
qtt = True

The particularity of this model is that we provide custom propensities for 2 of the reactions.
In the following, the reactions are defined and the CME generator is constructed.
The state is truncated to `512` per species to allow for the PMF to stay inside the domain as well as for quantization. 

In [ ]:
r1 = TTCME.ChemicalReaction(['A', 'B'], '->A', 3000.0, decomposable_propensity=[lambda x,val: 1.0, lambda x,val: 1/(val+x**2)],params=['v1'])
r2 = TTCME.ChemicalReaction(['A', 'B'], 'A->', 0.001)
r3 = TTCME.ChemicalReaction(['A', 'B'], '->B', 3000.0, decomposable_propensity=[lambda x,val: 1/(val+x**2), lambda x,val: 1.0],params=['v2'])
r4 = TTCME.ChemicalReaction(['A', 'B'], 'B->', 0.001)

mdl = TTCME.ReactionSystem(['A','B'],[r1, r2 ,r3, r4], params=['v1','v2'])
N = [100,100]

Nl = 64
param_range = [[9000,12000],[8000,13000]]
basis_param = [TTCME.basis.BSplineBasis(Nl,[p[0],p[1]],deg = 2) for p in param_range]
Att = mdl.generator_TT_parameters(N,[np.array([9000,11000]), np.array([9500,11000])])

The system is built for foxed parameters.

In [ ]:
r1t = TTCME.ChemicalReaction(['A', 'B'], '->A', 3000.0, decomposable_propensity=[lambda x: 1.0, lambda x: 1/(11000+x**2)])
r2t = TTCME.ChemicalReaction(['A', 'B'], 'A->', 0.001)
r3t = TTCME.ChemicalReaction(['A', 'B'], '->B', 3000.0, decomposable_propensity=[lambda x: 1/(11000+x**2), lambda x: 1.0])
r4t = TTCME.ChemicalReaction(['A', 'B'], 'B->', 0.001)

mdlt = TTCME.ReactionSystem(['A','B'],[r1t, r2t ,r3t, r4t])

Asp = mdlt.generator_sparse(N)

The generators are compared.

In [ ]:
Attt = Att[:,:,1,1,:,:,1,1]
print('Reltive error', np.linalg.norm(Asp.toarray()-Attt.numpy().reshape([100**2,-1]))/np.linalg.norm(Asp.toarray()))